In [1]:
import sys , os
import pandas as pd
sys.path.append(os.path.abspath('../scripts'))

In [12]:
import Modelling as mo

In [19]:


# Initialize the Modeling class
modeling = mo.Modeling()

# Load the data
path = '../data/proceced_data.csv'  
data = modeling.load_data(path)

data.head()



,Unnamed: 0,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,...,Transaction_Day,Transaction_Month,Transaction_Year,CurrencyCode_WOE,ProviderId_WOE,ProductId_WOE,ProductCategory_WOE,Recency,RFMS_score,Label
0,0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,256,5,1,...,15,11,2018,0.0,3.137005,1.645067,1.620379,2264,-0.042337,1
1,1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,256,3,19,...,15,11,2018,0.0,2.736867,4.189243,-0.565446,2264,-0.042337,1
2,2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,256,5,0,...,15,11,2018,0.0,3.137005,inf,1.620379,2264,-0.083785,0
3,3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,256,0,11,...,15,11,2018,0.0,-1.618899,-0.271621,-1.134962,2264,-0.068359,0
4,4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,256,3,19,...,15,11,2018,0.0,2.736867,4.189243,-0.565446,2264,-0.068359,0


In [26]:
%load_ext autoreload
%autoreload 2

import importlib
import Modelling

# Reload the updated module to ensure any changes are loaded
importlib.reload(Modelling)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'Modelling' from 'c:\\Users\\HP\\Documents\\Dagii\\Credit-scoring-model\\scripts\\Modelling.py'>

first load the data

In [21]:
# Preprocess the data
data = modeling.preprocess_data(data)
 # Inspect the transformed data


In [ ]:
# # Replace infinite values with NaN
# data_preprocessed.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Replace NaN values with the mean of the respective column
# data_preprocessed.fillna(data_preprocessed.mean(), inplace=True)

# # Verify there are no more infinite or very large values
# infinite_values_post = data_preprocessed.isin([np.inf, -np.inf]).sum()
# large_values_post = (data_preprocessed.abs() > np.finfo(np.float64).max).sum()

# print(f"Infinite values after cleaning: {infinite_values_post.sum()}")
# print(f"Very large values after cleaning: {large_values_post.sum()}")


Infinite values after cleaning: 0
Very large values after cleaning: 0


###Define Features and Target

###Initialize the Modeling Class and Split Data

In [22]:
X_train, X_test, y_train, y_test = modeling.split_the_data(data)

In [23]:
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

Training set size: 76529 samples
Testing set size: 19133 samples


## Why Choose Logistic Regression and Random Forest

### Logistic Regression
- **Simplicity and Interpretability**: Logistic Regression is straightforward and easy to interpret, making it an excellent choice for understanding the relationship between features and the target variable.
- **Efficiency**: It is computationally efficient, which is beneficial for large datasets.
- **Binary Classification**: Well-suited for binary classification tasks, providing probabilistic outputs that can be easily understood and interpreted.

### Random Forest
- **Robustness**: Combines multiple decision trees to reduce overfitting and improve generalization to unseen data.
- **Feature Importance**: Capable of evaluating the importance of each feature, providing insights into which features are most influential in making predictions.
- **Versatility**: Handles both classification and regression tasks effectively, making it a versatile choice for various problems.

### Combined Benefits
By using both Logistic Regression and Random Forest, you balance simplicity and interpretability with robustness and predictive performance. This combination allows you to compare and contrast different modeling techniques to find the best solution for your data.

## Summary
- **Logistic Regression**: Simple, interpretable, efficient for binary classification.
- **Random Forest**: Robust, reduces overfitting, evaluates feature importance.

This balanced approach provides a comprehensive understanding of your data and helps in making well-informed decisions based on the strengths of both models.


In [ ]:
# import numpy as np

# # Identify infinite or very large values in the test dataset
# infinite_values_test = X_test.isin([np.inf, -np.inf]).sum()
# large_values_test = (X_test.abs() > np.finfo(np.float64).max).sum()

# print(f"Infinite values in test data: {infinite_values_test.sum()}")
# print(f"Very large values in test data: {large_values_test.sum()}")

# # Replace infinite values with NaN
# X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Replace NaN values with the mean of the respective column
# X_test.fillna(X_test.mean(), inplace=True)

# # Verify there are no more infinite or very large values in the test data
# infinite_values_test_post = X_test.isin([np.inf, -np.inf]).sum()
# large_values_test_post = (X_test.abs() > np.finfo(np.float64).max).sum()

# print(f"Infinite values after cleaning in test data: {infinite_values_test_post.sum()}")
# print(f"Very large values after cleaning in test data: {large_values_test_post.sum()}")


Infinite values in test data: 2545
Very large values in test data: 2545
Infinite values after cleaning in test data: 0
Very large values after cleaning in test data: 0


In [ ]:
# # Replace infinite values with NaN
# data_preprocessed.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Replace NaN values with the mean of the respective column
# data_preprocessed.fillna(data_preprocessed.mean(), inplace=True)

# # Verify there are no more infinite or very large values
# infinite_values_post = data_preprocessed.isin([np.inf, -np.inf]).sum()
# large_values_post = (data_preprocessed.abs() > np.finfo(np.float64).max).sum()

# print(f"Infinite values after cleaning: {infinite_values_post.sum()}")
# print(f"Very large values after cleaning: {large_values_post.sum()}")


Infinite values after cleaning: 0
Very large values after cleaning: 0


In [27]:
# Train the models
logistic_model, random_forest_model = modeling.train_the_models(X_train, y_train,X_test)


TypeError: Modeling.train_the_models() takes 3 positional arguments but 4 were given

In [ ]:
logistic_model

In [ ]:
random_forest_model

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate Logistic Regression
print("Logistic Regression Evaluation:")
y_pred_logistic = modeling.evaluate_models(logistic_model, X_test, y_test)

# Evaluate Random Forest
print("\nRandom Forest Evaluation:")
y_pred_rf = modeling.evaluate_models(random_forest_model, X_test, y_test)


ERROR:Modelling:Error occurred while evaluating the model: 'NoneType' object has no attribute 'predict'
ERROR:Modelling:Error occurred while evaluating the model: 'NoneType' object has no attribute 'predict'


Logistic Regression Evaluation:

Random Forest Evaluation:


In [ ]:
# Check the distribution of the target variable
print(y_train.value_counts())

# If there's an imbalance, consider techniques like class weighting, oversampling, or SMOTE
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the Logistic Regression model with balanced data
logistic_model_smote = LogisticRegression(max_iter=1000, random_state=42)
logistic_model_smote.fit(X_train_smote, y_train_smote)

# Evaluate the model with balanced data
print("Logistic Regression with SMOTE Evaluation:")
y_pred_logistic_smote = modeling.evaluate_models(logistic_model_smote, X_test, y_test)


Label
0    38267
1    38262
Name: count, dtype: int64


NameError: name 'LogisticRegression' is not defined

In [ ]:
# Perform cross-validation to check for overfitting
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(random_forest_model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"Random Forest Cross-Validation ROC-AUC Scores: {cv_scores}")
print(f"Mean ROC-AUC: {cv_scores.mean():.4f}")


Random Forest Cross-Validation ROC-AUC Scores: [1. 1. 1. 1. 1.]
Mean ROC-AUC: 1.0000
